In [ ]:
import pandas as pd
from pulp import *
import random

# --- PARTE 1: FUNCIÓN DE OPTIMIZACIÓN DEL LINEUP ---
def optimizar_lineup(roster_df):
    """
    Función que implementa el modelo de Programación Lineal
    para encontrar el lineup óptimo.
    """
    # 1. Preparar el Problema
    prob = LpProblem("Fantasy_Lineup_Optimizer", LpMaximize)

    # 2. Variables de Decisión
    # Creamos una variable binaria (0 o 1) para cada jugador en el roster
    players = roster_df['Player'].tolist()
    player_vars = LpVariable.dicts("Select", players, 0, 1, LpBinary)

    # 3. Función Objetivo
    # Maximizar la suma de FPTS_PROYECTADOS de los jugadores seleccionados
    fp_projections = dict(zip(roster_df['Player'], roster_df['FPTS_PROYECTADOS']))
    prob += lpSum([fp_projections[p] * player_vars[p] for p in players]), "Total_FPTS"

    # 4. Restricciones del Lineup
    # Diccionarios para acceder rápidamente a los jugadores por posición
    pos_roster = roster_df.groupby('Position')['Player'].apply(list).to_dict()

    # A. Restricciones de Posición Fijas
    prob += lpSum([player_vars[p] for p in pos_roster.get('QB', [])]) == 1, "Num_QB"
    prob += lpSum([player_vars[p] for p in pos_roster.get('K', [])]) == 1, "Num_K"
    prob += lpSum([player_vars[p] for p in pos_roster.get('DST', [])]) == 1, "Num_DST"

    # B. Restricciones de Posición Mínimas (RB, WR, TE)
    prob += lpSum([player_vars[p] for p in pos_roster.get('RB', [])]) >= 2, "Min_RB"
    prob += lpSum([player_vars[p] for p in pos_roster.get('WR', [])]) >= 2, "Min_WR"
    prob += lpSum([player_vars[p] for p in pos_roster.get('TE', [])]) >= 1, "Min_TE"

    # C. Restricción del Flex y Tamaño Total (6 jugadores en total entre RB, WR, TE)
    FLEX_POS = ['RB', 'WR', 'TE']
    flex_players = [p for pos in FLEX_POS for p in pos_roster.get(pos, [])]
    # Suma de jugadores en RB, WR, TE debe ser 6 (2RB + 2WR + 1TE + 1FLEX)
    prob += lpSum([player_vars[p] for p in flex_players]) == 6, "Total_RB_WR_TE_and_FLEX"
    
    # D. Restricción del Total de Titulares (9 posiciones fijas + 1 Flex = 10)
    prob += lpSum([player_vars[p] for p in players]) == 10, "Total_Starters"

    # 5. Resolver el Problema
    prob.solve()

    # 6. Extracción de Resultados
    starters = []
    bench = []
    
    if prob.status == LpStatusOptimal:
        for p in players:
            if player_vars[p].varValue == 1:
                starters.append(roster_df[roster_df['Player'] == p].iloc[0].to_dict())
            else:
                bench.append(roster_df[roster_df['Player'] == p].iloc[0].to_dict())

        return pd.DataFrame(starters), pd.DataFrame(bench), value(prob.objective)
    else:
        return None, None, f"Estado de la solución: {LpStatus[prob.status]}"

In [ ]:
# --- PARTE 2: SIMULACIÓN DE MOCK DRAFT Y EJEMPLO ---

def simular_mock_draft(player_pool_df):
    """
    Simula un mock draft de 14 rondas para un solo equipo,
    seleccionando a los mejores jugadores disponibles por ronda.
    """
    # 1. Clasificar jugadores por FPTS proyectados (el mejor es el Rank 1)
    draft_pool = player_pool_df.sort_values(by='FPTS_PROYECTADOS', ascending=False).copy()
    
    roster_final = []
    ronda = 1
    
    print("\n--- 📝 SIMULACIÓN DE MOCK DRAFT (14 RONDAS) ---")
    
    for _ in range(14):
        if draft_pool.empty:
            break
            
        # El equipo siempre toma al jugador con el FPTS más alto disponible
        pick = draft_pool.iloc[0].copy()
        
        roster_final.append(pick)
        
        print(f"Ronda {ronda:02d}: {pick['Player']} ({pick['Position']}) - {pick['FPTS_PROYECTADOS']:.2f} FPTS")
        
        # Eliminar al jugador seleccionado del pool de draft
        draft_pool = draft_pool.iloc[1:]
        ronda += 1
        
    return pd.DataFrame(roster_final)

In [ ]:
# --- PARTE 3: PREPARACIÓN DE DATOS (MOCK DATA) ---

# Crear un DataFrame de ejemplo que imita la salida de tu EDA/Scraping
def crear_mock_data():
    # Jugadores de ejemplo con proyecciones arbitrarias (FPTS_PROYECTADOS)
    data = {
        'Player': [
            'P. Mahomes', 'J. Allen', 'T. Kelce', 'L. Jackson', 'J. Jefferson',
            'C. McCaffrey', 'T. Hill', 'S. Barkley', 'D. Adams', 'B. Hall',
            'J. Waddle', 'D. Waller', 'D. Montgomery', 'C. Akers', 'I. Smith',
            'J. Meyers', 'B. Uzomah', 'J. Moody', 'Eagles D/ST', 'C. Kmet'
        ],
        'Position': [
            'QB', 'QB', 'TE', 'QB', 'WR', 
            'RB', 'WR', 'RB', 'WR', 'RB', 
            'WR', 'TE', 'RB', 'RB', 'TE', 
            'WR', 'TE', 'K', 'DST', 'TE'
        ],
        'FPTS_PROYECTADOS': [
            280.5, 275.2, 210.1, 260.0, 205.0, 
            250.0, 195.0, 220.0, 180.0, 175.0, 
            160.0, 150.0, 140.0, 130.0, 120.0, 
            110.0, 90.0, 115.0, 125.0, 80.0
        ]
    }
    return pd.DataFrame(data)

In [ ]:
# --- PARTE 4: EJECUCIÓN DEL FLUJO ---

# 1. Obtener datos de jugadores (en tu proyecto, esto vendría de tu scraping/EDA)
player_pool_df = crear_mock_data()

# 2. Simular el Mock Draft (selecciona 14 jugadores para el roster)
roster_df = simular_mock_draft(player_pool_df)

print("\n" + "="*80)
print(f"ROSTER FINAL DEL EQUIPO (14 jugadores):")
print("="*80)
print(roster_df[['Player', 'Position', 'FPTS_PROYECTADOS']].to_string(index=False))

# 3. Optimizar el Lineup con el roster generado
starters_df, bench_df, total_fpts = optimizar_lineup(roster_df)

print("\n" + "="*80)

if isinstance(starters_df, pd.DataFrame):
    print(f"✅ LINEUP ÓPTIMO GENERADO | Puntuación Total Proyectada: {total_fpts:.2f} FPTS")
    print("="*80)
    
    # Asegurar que el Flex se muestre correctamente si es necesario
    # Una vez resuelto por PuLP, la asignación automática por posición (2RB, 2WR, 1TE)
    # y el total de 6 entre ellos define el FLEX.

    starters_df = starters_df.sort_values(by='FPTS_PROYECTADOS', ascending=False)
    
    # Crear un resumen visual de la alineación
    print("\n--- 🏈 STARTING LINEUP (10 Jugadores) ---")
    
    # Asignación de Posiciones Fijas
    print(f"| QB:  {starters_df[starters_df['Position'] == 'QB'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'QB'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    print(f"| RB1: {starters_df[starters_df['Position'] == 'RB'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'RB'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    print(f"| RB2: {starters_df[starters_df['Position'] == 'RB'].iloc[1]['Player']} ({starters_df[starters_df['Position'] == 'RB'].iloc[1]['FPTS_PROYECTADOS']:.2f})")
    print(f"| WR1: {starters_df[starters_df['Position'] == 'WR'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'WR'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    print(f"| WR2: {starters_df[starters_df['Position'] == 'WR'].iloc[1]['Player']} ({starters_df[starters_df['Position'] == 'WR'].iloc[1]['FPTS_PROYECTADOS']:.2f})")
    print(f"| TE:  {starters_df[starters_df['Position'] == 'TE'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'TE'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    print(f"| K:   {starters_df[starters_df['Position'] == 'K'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'K'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    print(f"| DST: {starters_df[starters_df['Position'] == 'DST'].iloc[0]['Player']} ({starters_df[starters_df['Position'] == 'DST'].iloc[0]['FPTS_PROYECTADOS']:.2f})")
    
    # Determinar el jugador FLEX: el jugador seleccionado restante entre RB/WR/TE
    rb_selected = starters_df[starters_df['Position'] == 'RB'].iloc[0:2]
    wr_selected = starters_df[starters_df['Position'] == 'WR'].iloc[0:2]
    te_selected = starters_df[starters_df['Position'] == 'TE'].iloc[0:1]
    
    selected_fixed = pd.concat([rb_selected, wr_selected, te_selected])
    
    # Buscar el jugador FLEX (el que está en starters_df pero no en selected_fixed)
    flex_player = starters_df[~starters_df.apply(tuple,1).isin(selected_fixed.apply(tuple,1))].sort_values(by='FPTS_PROYECTADOS', ascending=False).iloc[0]
    
    print(f"| FLEX: {flex_player['Player']} ({flex_player['Position']}) ({flex_player['FPTS_PROYECTADOS']:.2f})")
    
    # Mostrar la Banca
    print("\n--- 🪑 JUGADORES EN LA BANCA (5 Jugadores) ---")
    print(bench_df[['Player', 'Position', 'FPTS_PROYECTADOS']].sort_values(by='FPTS_PROYECTADOS', ascending=False).to_string(index=False))

else:
    print(f"❌ ERROR: No se pudo encontrar una solución óptima. {total_fpts}")